# Object Detection Model

This notebook explains how the model is designed and how to use it.  

The model based on a [Faster RCNN model with a ResNet-50-FPN backbone](https://pytorch.org/vision/stable/models/generated/torchvision.models.detection.fasterrcnn_resnet50_fpn.html#torchvision.models.detection.fasterrcnn_resnet50_fpn). The input to the model is a list of tensors, each of shape [Color, Width, Height], one for each image.  

The trained model is able to detect circles and lines on an image, in this case these represent holes and scratches on a picture of a piece of work from the shopfloor. This way a postprocessing step is able to decide if the piece is intact and ready for further processing.  
The images for testing the model can be produced with notebook [01-CreateTestSet.ipynb](./01-CreateTestSet.ipynb).  
The model is placed in the `models` folder with name [`object_detection.onnx`](../models/object_detection.onnx).

In [ ]:
import onnx
model_onnx = onnx.load("../models/object_detection.onnx")

Once the model is loaded we can study it with its input and outputs.  
As that is shown below, the graph was saved with in input tensor with shape `[1 x 3 x 224 x 224]`.  

In [ ]:
model_onnx.graph.input

Similarly to the input, we also can explore the output shape of the model:

In [ ]:
model_onnx.graph.output

Here we can spot, that the model will produce three outputs, one for the coordinates of `boxes` found, one for the `labels` which object was found in the box and another outputs with the `scores`, showing how sure the model on the prediction.  
We also can spot that their first dimension is not defined, as we don't know, how many object will be spotted on the picture.
The case of `GPU Runtime` of `AI Inference Server`, this dimension should be defined with dimension `-1` instead of leaving it undefined.  
So the next cell defines these dimensions for `-1`.

In [ ]:
for output in model_onnx.graph.output:
    print("before\n", output.type.tensor_type.shape)
    for dim in output.type.tensor_type.shape.dim:
        if dim.dim_value == 0:
            print("now")
            dim.dim_value = -1
    print("after\n", output.type.tensor_type.shape)
onnx.save(model_onnx, "../src/detection/1/model.onnx")

## Load the model for inference

If we want to validate the model, we need to initiate an [InferenceSession](https://onnxruntime.ai/docs/api/python/api_summary.html) by [onnxruntime]() module.  
This will enable the model to run and produce outputs from the given inputs.

In [ ]:
from onnxruntime import InferenceSession

session = InferenceSession("../src/detection/1/model.onnx")

## Input tensors

In the code block below we create a method which can visualize the predictions for a single image.  
By defining variable `CONFIDENCE` we set a limit, so only the predictions will be visualized where the confidence of the prediction is higher than this value.

In [ ]:
import numpy
from PIL import ImageDraw, ImageFont

CLASSES = [ '__background__', 'hole', 'scratch' ]
COLORS = list(tuple(x) for x in numpy.random.randint(low=0, high=255, size=(len(CLASSES), 3)))
_font = ImageFont.load_default(size=11)

_GREEN = (16, 255, 16)  # color for hole boxes
_RED = (255, 16, 16)  # color for scratch boxes

CONFIDENCE = 0.8

def draw_prediction(image, boxes, labels, scores):
	canvas = ImageDraw.Draw(image)
	for i in range(0, len(boxes)):
		_class = labels[i]
		confidence = scores[i]
		if confidence > CONFIDENCE:
			box = boxes[i]
			(startY, startX, endY, endX) = box.astype("int") 
			label = "{}: {:.2f}%".format(CLASSES[_class], confidence * 100)
			print("[INFO] {} ({})".format(label, box))
			COLOR = _GREEN if _class == 1 else _RED
			canvas.rectangle([(startX, startY), (endX, endY)], outline=COLOR, width=2)

			y = startY - 4 if startY - 4 > 4 else startY + 4
			canvas.text(xy=(startX+4, y), text=label, font=_font)
	

By executing the code block below, it will choose images randomly in  number of `samples_size` and then visualize the predictions.  
The `CONFIDENCE` threshold can be set as described above.

In [ ]:
from pathlib import Path
from PIL import Image

image_dir = Path('../data/processed')

images = list(image_dir.rglob("./**/*.jpg"))
images_count = len(images)

samples_size = 3
sample_indexes = set(numpy.random.randint(0, images_count - 1, (samples_size))) 
print("sample_indexes", sample_indexes)

CONFIDENCE = 0.5

sample_images = []
for idx in sample_indexes:
    print(f"loading image '{images[idx]}'..")
    pil_image = Image.open(images[idx])
    image_array = numpy.array(pil_image).astype('float32').transpose(2,1,0)
    image_array /= 255.
    
    session = InferenceSession("../src/detection/1/model.onnx")
    boxes, labels, scores = session.run(["boxes","labels","scores"], {"input": numpy.array([image_array])})
    draw_prediction(pil_image, boxes, labels, scores)
    pil_image.show()

Now we are ready to create 

- a `Preprocessing` step to create the `input` for the GPURuntimeComponent, and
- a `Postprocessing` step to process the `output` of the GPURuntimeComponent  

in notebook [20-PreAndPostProcessing.ipynb](./20-PreAndPostProcessing.ipynb)